In [1]:
# Adjust the settings in the next cell if necessary

In [2]:
import pytesseract

# COMMENT if NOT using windows (i.e., add # before both lines)
pytesseract.pytesseract.tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract"
TESSDATA_PREFIX = r"C:\Program Files\Tesseract-OCR"

# UNCOMMENT if using latest tesseract version on mac via 'brew install tesseract'
#pytesseract.pytesseract.tesseract_cmd = "/usr/local/Cellar/tesseract/4.1.1/bin/tesseract"

# Change to False if you always want your actual main stat value
assume_max_lv_gear = True

In [3]:
# Do not change anything below here unless you know what you are doing

In [4]:
from PIL import Image
from pytesseract import image_to_string
from IPython.display import display

def process(image):
    resize = cv2.resize(image, (0,0), fx=5, fy=5)
    gray = cv2.cvtColor(resize, cv2.COLOR_BGR2GRAY) # convert to gray for thresholding + blurring
    thresh = cv2.threshold(gray, 70, 255, cv2.THRESH_BINARY_INV)[1]
    blur = cv2.medianBlur(thresh, 3)
    color = cv2.cvtColor(blur, cv2.COLOR_GRAY2RGB) # convert to back to color
    color_img = Image.fromarray(color) # convert from array back into image format
    config = '--psm 6 --oem 1' # PSM 6 = treat as uniform block of text
    data = image_to_string(color_img, lang='eng', config=config)
    return data

def iterative_process(image):
    # Since level and plus are read off the gear icon, there is a lot of noise
    # Thus, we need to use an iterative process to try different thresholding settings to extract a value from them
    resize = cv2.resize(image, (0,0), fx=5, fy=5)
    gray = cv2.cvtColor(resize, cv2.COLOR_RGB2GRAY) # convert to gray for thresholding + blurring
    for low in [80,100,120,140,160]:
        thresh = cv2.threshold(gray, low, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)[1]
        blur = cv2.medianBlur(thresh, 3)
        color = cv2.cvtColor(blur, cv2.COLOR_GRAY2RGB) # convert to back to color
        color_img = Image.fromarray(color) # convert from array back into image format
        config = '--psm 8 --oem 1 -c tessedit_char_whitelist=0123456789' # PSM 8 = treat as single word / whitelist only allows digits
        data = image_to_string(color_img, lang='eng', config=config)
        #display(color_img)
        #print(low,data)
        if any(i.isdigit() for i in data): return data # If it finds a value it will terminate the loop and return the value
    return '1' # return '1' if the OCR fails to recognize the level or plus, so that there are no import errors

def stat_converter(stat):
    result = '' # default value to return just in case the stat data is bad
    if 'attack' in stat.lower():
        result = 'Atk'
        if '%' in stat: result += 'P'
    elif 'health' in stat.lower():
        result = 'HP'
        if '%' in stat: result += 'P'
    elif 'defense' in stat.lower():
        result = 'Def'
        if '%' in stat: result += 'P'
    elif 'speed' in stat.lower():
        result = 'Spd'
    elif 'chance' in stat.lower():
        result = 'CChance'
    elif 'damage' in stat.lower():
        result = 'CDmg'
    elif 'effectiveness' in stat.lower():
        result = 'Eff'
    elif 'resistance' in stat.lower():
        result = 'Res'
    return result
  

def digit_filter(val):
    # This attempts to filter out all characters that are not digits
    # If there are no digits, then it returns 0
    try: return int(''.join(filter(str.isdigit,val)))
    except ValueError: return 0

def char_filter(val):
    # This attempts to filter out all characters that are not letters
    # If there are no letters, then it returns ''
    try: return ''.join(filter(str.isalpha,val)).capitalize()
    except ValueError: return ''

def max_stat(data,item):
    stat = stat_converter(data)
    val = digit_filter(data) # Begin by setting val = actual value so that it gets returned if the rest fails
    if item['ability'] < 15: # Only change stats on items where they need to be increased
        if item['level'] in range(58,73):
            if stat == 'CChance': val = 45
            elif stat == 'CDmg': val = 55
            elif stat == 'Spd': val = 35
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 50
            elif stat == 'HP': val = 2295 # Not exactly right, finer-grained scaling
            elif stat == 'Def': val = 250 # Not exactly right, finer-grained scaling
            elif stat == 'Atk': val = 425 # Not exactly right, finer-grained scaling
        elif item['level'] in range(74,86):
            if stat == 'CChance': val = 55
            elif stat == 'CDmg': val = 65
            elif stat == 'Spd': val = 40
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 60
            elif stat == 'HP': val = 2700 # Not exactly right, finer-grained scaling
            elif stat == 'Def': val = 300 # Not exactly right, finer-grained scaling
            elif stat == 'Atk': val = 500 # Not exactly right, finer-grained scaling
        elif item['level'] in range(87,89):
            if stat == 'CChance': val = 60
            elif stat == 'CDmg': val = 70
            elif stat == 'Spd': val = 45
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 65
            elif stat == 'HP': val = 2765
            elif stat == 'Def': val = 310
            elif stat == 'Atk': val = 515
    return val

In [5]:
from glob import glob
from string import ascii_lowercase, digits
import random
import json
import cv2

# Format the json for the optimizer
export = {'processVersion':'1','heroes':[],'items':[]}
temp_list = []

print('Beginning export process...')

# Gather the filenames for all the screenshots. Looks for png, jpg, and jpeg.
filenames = glob('screenshots/*.png')+glob('screenshots/*.jpg')+glob('screenshots/*.jpeg')

did_break = False
errors = 0
for name in filenames:
    
    # The height of the item box changes depending on the length of the item and set descriptions,
    # we have to crop the top and bottom info separately in order to ensure the OCR boxes within these areas
    # remain in fixed locations. we then process the top and bottom info independently.
    
    print('currently processing '+name.replace('screenshots\\',''))
    screenshot = cv2.imread(name)
    
    # Make sure the screenshot has the proper dimensions
    if screenshot.shape[0] != 1080 or screenshot.shape[1] != 2200:
        print('ERROR: This screenshot is not 2200x1080!')
        print('Please retake the screenshot in Nox or LDPlayer with the internal resolution set to 2200x1080')
        did_break = True
        break
    
    # Setup dictionary for the current item
    item = {'locked':False,'efficiency':0}
    
    # Settings for how the boxes are cropped. If the UI changes, these may need to be updated
    width = [725,1190] # Same width for both boxes. Based on width of gearbox
    top_depth = 160 # This is how many pixels deep the top cropped image will be
    bottom_offset = 25 # This is how many pixels we start the bottom box below the divider in the gear screenshot
    bottom_depth = 340 # This is how many pixels deep the bottom cropped image will be
    
    # Top box
    template_top = cv2.imread('e7/top.jpg',0)
    gray = cv2.cvtColor(screenshot, cv2.COLOR_BGR2GRAY) # convert screenshot to grayscale
    match = cv2.matchTemplate(gray, template_top, cv2.TM_CCOEFF_NORMED) # find the template on the screenshot
    a, b, c, max_loc = cv2.minMaxLoc(match) # extract the location of the match (max_loc), tossing the rest
    X1 = max_loc[1]
    X2 = max_loc[1]+top_depth
    Y1 = width[0]
    Y2 = width[1]
    top_box = screenshot[X1:X2,Y1:Y2] # crop the box at those pixel locations
    #display(Image.fromarray(top_box)) # uncomment to see what the top_box looks like
    #Image.fromarray(top_box).save("top_box.png") # uncomment to export top_box as png

    # Bottom box
    template_bot = cv2.imread('e7/bottom.jpg',0)
    gray = cv2.cvtColor(screenshot, cv2.COLOR_BGR2GRAY) # convert screenshot to grayscale
    match = cv2.matchTemplate(gray, template_bot, cv2.TM_CCOEFF_NORMED) # find the template on the screenshot
    a, b, c, max_loc = cv2.minMaxLoc(match) # extract the location of the match (max_loc), tossing the rest
    X1 = max_loc[1]+bottom_offset
    X2 = max_loc[1]+bottom_offset+bottom_depth
    Y1 = width[0]
    Y2 = width[1]
    bottom_box = screenshot[X1:X2,Y1:Y2] # crop the box at those pixel locations
    #display(Image.fromarray(bottom_box)) # uncomment to see what the bottom_box looks like
    #Image.fromarray(bottom_box).save("bottom_box.png") # uncomment to export bottom_box as png
    
    # Coordinates for the info inside the top_box. If the UI changes, these may need to be updated
    # Coordinates are *relative* to the dimensions of top_box
    # Format: stat = [X1,X2,Y1,Y2] as this is the format used for slicing images
    types = [20,71,172,432]
    level = [19,47,35,69]
    plus = [0,28,132,161]
    
    # Process stats from the Top box
    # Type
    type_img = top_box[types[0]:types[1],types[2]:types[3]]
    #display(Image.fromarray(type_img)) # uncomment to see what the type_img looks like
    #Image.fromarray(type_img).save("type_img.png") # uncomment to export type_img as png
    data = process(type_img)
    split_data = data.split(' ')
    item['rarity'] = char_filter(split_data[0])
    item['slot'] = char_filter(split_data[1].split('\n')[0])
    if len(item['rarity']) == 0: print('Error: no rarity detected')
    if len(item['slot']) == 0: print('Error: no slot detected')
    
    # Level
    lvl_img = top_box[level[0]:level[1],level[2]:level[3]]
    #display(Image.fromarray(lvl_img)) # uncomment to see what the lvl_img looks like
    #Image.fromarray(lvl_img).save("lvl_img.png") # uncomment to export lvl_img as png
    data = iterative_process(lvl_img)
    data = data.replace('S','5').replace('B','8').replace('a','8') # Fix common OCR errors
    item['level'] = digit_filter(data)
    
    # Enhance lvl (plus)
    plus_img = top_box[plus[0]:plus[1],plus[2]:plus[3]]
    #display(Image.fromarray(plus_img)) # uncomment to see what the plus_img looks like
    #Image.fromarray(plus_img).save("plus_img.png") # uncomment to export plus_img as png
    data = iterative_process(plus_img)
    data = data.replace('S','5').replace('B','8').replace('a','8') # Fix common OCR errors
    item['ability'] = digit_filter(data)

    print("ilvl", item['level'],"/ enhanced to +"+str(item['ability'])+" / slot:",item['slot']," / rarity:",item['rarity'])
    
    # Coordinates for the info inside the bottom_box. If the UI changes, these may need to be updated
    # Coordinates are *relative* to the dimensions of bottom_box
    # Format: stat = [X1,X2,Y1,Y2] as this is the format used for slicing images
    main = [8,70,65,435]
    subs = [98,255,25,435]
    sets = [280,340,76,435]
    
    # Process stats from Bottom box    
    # Main
    main_img = bottom_box[main[0]:main[1],main[2]:main[3]]
    #display(Image.fromarray(main_img)) # uncomment to see what the main_img looks like
    #Image.fromarray(main_img).save("main_img.png") # uncomment to export main_img as png
    data = process(main_img)
    stat = stat_converter(data)
    if assume_max_lv_gear is True: item['mainStat'] = [stat,max_stat(data,item)]
    else: item['mainStat'] = [stat,digit_filter(data)]
    
    # Subs
    subs_img = bottom_box[subs[0]:subs[1],subs[2]:subs[3]]
    #display(Image.fromarray(subs_img)) # uncomment to see what the subs_img looks like
    #Image.fromarray(subs_img).save("subs_img.png") # uncomment to export subs_img as png
    data = process(subs_img)
    for n,entry in enumerate(data.split('\n')):
        if n < 4:
            stat = stat_converter(entry)
            entry = entry.replace('T','7') # Fix common OCR error
            val = digit_filter(entry)
            if len(stat) > 0 and val != 0:
                item['subStat'+str(n+1)] = [stat,val]
    
    # Set
    set_img = bottom_box[sets[0]:sets[1],sets[2]:sets[3]]
    #display(Image.fromarray(set_img)) # uncomment to see what the set_img looks like
    #Image.fromarray(set_img).save("set_img.png") # uncomment to export set_img as png
    data = process(set_img)
    data_split = data.split(' Set')
    item['set'] = char_filter(data_split[0])
    if len(item['set']) == 0: print('Error: no set detected')
    
    print("set:", item['set']," / main stat:",item['mainStat'])
    
    # Check to make sure the item does not already exist in the item dictionary
    # Also verifies that it has a valid slot and rarity entry
    # If these conditions are met, this assigns the item a unique ID and adds it to the item dictionary for export
    if item not in temp_list and len(item['slot']) > 0 and len(item['rarity']) > 0:
        temp_list.append(item)
        item['id'] = 'jt'+''.join(random.choice(digits+ascii_lowercase) for _ in range(6))
        export['items'].append(item)
        print(len(export['items']),"exported out of",len(filenames)-errors,'valid items \n')
    else:
        errors += 1
        print('Item not exported because of fatal errors in processing it \n')

# Export dictionary to json for importing into optimizer if everything completed properly
if not did_break:
    with open('exported_gear.json', 'w') as f: json.dump(export, f)
    print('JSON file finished!')

Beginning export process...
currently processing screenshots/100149477-19814e80-2e53-11eb-9083-f89fc4615407.png
ilvl 71 / enhanced to +15 / slot: Weapon  / rarity: Epic
set: Rage  / main stat: ['Atk', 440]
1 exported out of 8 valid items 

currently processing screenshots/test_screenshot.png
ilvl 88 / enhanced to +15 / slot: Weapon  / rarity: Epic
set: Speed  / main stat: ['Atk', 515]
2 exported out of 8 valid items 

currently processing screenshots/100166642-0631ab80-2e72-11eb-9281-cd708f834116.png
ilvl 90 / enhanced to +15 / slot: Helmet  / rarity: Epic
set: Critical  / main stat: ['HP', 2835]
3 exported out of 8 valid items 

currently processing screenshots/100166640-05991500-2e72-11eb-9835-051053694fb6.png
ilvl 90 / enhanced to +15 / slot: Helmet  / rarity: Epic
set: Speed  / main stat: ['HP', 2835]
4 exported out of 8 valid items 

currently processing screenshots/100166631-02058e00-2e72-11eb-906c-11cadc470078.png
ilvl 85 / enhanced to +15 / slot: Necklace  / rarity: Heroic
set: